After another prod run, some analytics


# Install Required Libraries
Use pip to install necessary libraries such as psycopg2 or sqlalchemy.

In [42]:
# Install Required Libraries
# Uncomment the following lines to install the required libraries
# !pip install psycopg2-binary
# !pip install sqlalchemy

# Import Libraries
Import libraries like psycopg2, sqlalchemy, and pandas for database connection and data manipulation.

In [43]:
# Import Libraries - Enhanced for Statistical Analysis
import psycopg2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Set up plotting style
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)

# Set Up Database Connection
Establish a connection to the PostgreSQL database using connection parameters such as host, database name, user, and password.

In [44]:
# Set Up Database Connection
# Replace the placeholders with your actual database credentials
DB_CONFIG = {
    "dbname": "csgo_parsed",
    "user": "csgo_parser",
    "password": "3?6B7yTGPrkJF34p",
    "host": "192.168.1.100",
    "port": "5444"
}

# Using psycopg2
try:
    connection = psycopg2.connect(**DB_CONFIG)
    print("Connection to PostgreSQL database successful")
except Exception as e:
    print(f"Error: {e}")



Connection to PostgreSQL database successful


# Query the PostgreSQL Table
Write and execute a SQL query to fetch data from the desired table.

# CS:GO Descriptive Statistics Analysis

This section provides comprehensive descriptive statistics covering:
- Player Economic Analysis (Spending, Earning, Equipment Values)
- Team Economic Performance 
- Kill/Death/Assist Statistics
- Weapon Analysis and Efficiency
- Inventory Composition
- Win/Loss Streaks and Patterns
- Bomb Plant/Defuse Outcomes
- Regression Analysis on Round Outcomes

In [45]:
# Establish connection and load core datasets for analysis
import warnings
warnings.filterwarnings('ignore')

# Ensure we have a fresh connection
try:
    connection = psycopg2.connect(**DB_CONFIG)
    print("✅ Database connection established")
except Exception as e:
    print(f"❌ Connection error: {e}")

def execute_query(query, description=""):
    """Execute query and return pandas DataFrame"""
    try:
        cursor = connection.cursor()
        cursor.execute(query)
        data = cursor.fetchall()
        columns = [desc[0] for desc in cursor.description]
        df = pd.DataFrame(data, columns=columns)
        cursor.close()
        if description:
            print(f"✅ {description}: {len(df):,} records loaded")
        return df
    except Exception as e:
        print(f"❌ Error in {description}: {e}")
        return pd.DataFrame()

# Load ALL demo data for comprehensive analysis
print("📊 Loading comprehensive datasets for ALL demoes analysis...")

# Get ALL demoes for comprehensive analysis - NO LIMITS
all_demoes_query = """
SELECT 
    id_demo_exports as id_demo_exports, 
    event_id, 
    COUNT(*) as rounds_count,
    MIN(id) as first_round_id,
    MAX(id) as last_round_id
FROM rounds_ed 
WHERE id_demo_exports IS NOT NULL 
GROUP BY id_demo_exports, event_id
ORDER BY id_demo_exports DESC
"""

all_demoes = execute_query(all_demoes_query, "ALL demoes in database")

print(f"🎯 COMPREHENSIVE ANALYSIS across ALL {len(all_demoes):,} demoes")
print(f"Total rounds available: {all_demoes['rounds_count'].sum():,}")
print(f"Average rounds per demo: {all_demoes['rounds_count'].mean():.1f}")
print(f"demo ID range: {all_demoes['id_demo_exports'].min()} - {all_demoes['id_demo_exports'].max()}")

# Sample a few demo IDs for display but analyze ALL for statistics
sample_demoes = all_demoes['id_demo_exports'].tolist()[:5]
print(f"Sample demo IDs: {sample_demoes}")
print("📈 All subsequent queries will analyze the COMPLETE dataset!")

✅ Database connection established
📊 Loading comprehensive datasets for ALL demoes analysis...
✅ ALL demoes in database: 98,937 records loaded
🎯 COMPREHENSIVE ANALYSIS across ALL 98,937 demoes
Total rounds available: 2,601,592
Average rounds per demo: 26.3
demo ID range: 1 - 242827
Sample demo IDs: [242827, 242826, 242825, 234825, 234815]
📈 All subsequent queries will analyze the COMPLETE dataset!
✅ ALL demoes in database: 98,937 records loaded
🎯 COMPREHENSIVE ANALYSIS across ALL 98,937 demoes
Total rounds available: 2,601,592
Average rounds per demo: 26.3
demo ID range: 1 - 242827
Sample demo IDs: [242827, 242826, 242825, 234825, 234815]
📈 All subsequent queries will analyze the COMPLETE dataset!


## 1. Player Economic Analysis

In [46]:
# Player Economic Statistics: Spending, Earning, Equipment Values - ALL demoES

# Player spending analysis - CORRECTED for actual schema + ALL demoES
player_economics_query = """
SELECT 
    pr.player_id,
    pr.team,
    COUNT(*) as rounds_played,
    -- Economic metrics from player_economy_ed
    AVG(pe.money_spent) as avg_money_spent,
    SUM(pe.money_spent) as total_spent,
    AVG(pe.money_earned) as avg_money_earned,
    AVG(pe.saved_eq_val) as avg_saved_equipment,
    AVG(pe.starting_eq_val) as avg_starting_equipment,
    STDDEV(pe.money_spent) as spending_volatility,
    -- Performance metrics from player_round_ed
    Max(pr.kills) as avg_kills,
    Max(pr.deaths) as avg_deaths,
    Max(pr.assists) as avg_assists,
    -- Equipment values at different points
    AVG(pr.eq_val_fte) as avg_equipment_fte,
    AVG(pr.eq_val_re) as avg_equipment_re,
    -- Additional comprehensive metrics
    COUNT(DISTINCT r.id_demo_exports) as demoes_played
FROM player_round_ed pr
JOIN player_economy_ed pe ON pr.id = pe.player_round_id
JOIN rounds_ed r ON pr.round_id = r.id
WHERE pe.money_spent IS NOT NULL
GROUP BY pr.player_id, pr.team
HAVING COUNT(*) >= 10  -- Minimum 10 rounds for meaningful statistics
ORDER BY avg_money_spent DESC
"""

player_economics = pd.read_sql(player_economics_query, connection)

# Calculate derived metrics
player_economics['savings_efficiency'] = (player_economics['avg_saved_equipment'] / player_economics['avg_money_spent'].replace(0, np.nan)).fillna(0)
player_economics['kd_ratio'] = (player_economics['avg_kills'] / player_economics['avg_deaths'].replace(0, 1)).fillna(0)
player_economics['spending_efficiency'] = (player_economics['avg_equipment_fte'] / player_economics['avg_money_spent']).fillna(0)

# Summary statistics - COMPREHENSIVE
print("=== COMPREHENSIVE PLAYER ECONOMIC SUMMARY (ALL demoES) ===")
print(f"Players analyzed: {len(player_economics):,}")
print(f"Total rounds analyzed: {player_economics['rounds_played'].sum():,}")
print(f"Total demoes covered: {player_economics['demoes_played'].sum():,}")
print(f"Average spending per round: ${player_economics['avg_money_spent'].mean():.0f}")
print(f"Median spending per round: ${player_economics['avg_money_spent'].median():.0f}")
print(f"Spending standard deviation: ${player_economics['avg_money_spent'].std():.0f}")
print(f"Top 10% spenders threshold: ${player_economics['avg_money_spent'].quantile(0.9):.0f}")
print(f"Top 1% spenders threshold: ${player_economics['avg_money_spent'].quantile(0.99):.0f}")

# Display comprehensive statistics
print(f"\n=== COMPREHENSIVE PERFORMANCE METRICS ===")
print(f"Average KD ratio across all players: {player_economics['kd_ratio'].mean():.3f}")

player_economics.describe()

=== COMPREHENSIVE PLAYER ECONOMIC SUMMARY (ALL demoES) ===
Players analyzed: 23,120
Total rounds analyzed: 24,896,913
Total demoes covered: 989,210
Average spending per round: $2603
Median spending per round: $2628
Spending standard deviation: $189
Top 10% spenders threshold: $2798
Top 1% spenders threshold: $2995

=== COMPREHENSIVE PERFORMANCE METRICS ===
Average KD ratio across all players: 0.980


,player_id,team,rounds_played,avg_money_spent,total_spent,avg_money_earned,avg_saved_equipment,avg_starting_equipment,spending_volatility,avg_kills,avg_deaths,avg_assists,avg_equipment_fte,avg_equipment_re,demoes_played,savings_efficiency,kd_ratio,spending_efficiency
count,23120.000000,23120.000000,23120.000000,23120.000000,2.312000e+04,23120.000000,23120.000000,23120.000000,23120.000000,23120.000000,23120.000000,23120.00000,23120.000000,23120.000000,23120.000000,23120.000000,23120.000000,23120.000000
mean,15172.447275,1.538581,1076.856099,2603.411404,2.861015e+06,2788.978835,1187.355845,1281.599588,1948.771748,24.619377,24.846453,6.45346,3730.023449,3913.238760,42.785900,0.456785,0.980046,1.433999
std,5297.712142,0.498520,2543.113770,188.832756,6.784695e+06,182.976449,438.808118,404.647058,157.236584,10.035433,6.861406,3.38594,459.029812,417.540418,100.148741,0.175082,0.293444,0.161210
min,0.000000,1.000000,15.000000,808.823529,1.375000e+04,1250.000000,0.000000,200.000000,970.218290,0.000000,3.000000,0.00000,1259.375000,2276.666667,1.000000,0.000000,0.000000,0.773438
25%,11617.750000,1.000000,49.000000,2506.299213,1.274500e+05,2699.115930,947.614538,1060.469405,1853.935228,17.000000,20.000000,4.00000,3484.961286,3698.378800,2.000000,0.362920,0.800000,1.346769
50%,15530.500000,2.000000,139.000000,2627.839681,3.624250e+05,2820.254183,1228.482702,1319.588675,1935.132688,24.000000,23.000000,6.00000,3772.203453,3939.334030,6.000000,0.465870,1.000000,1.429641
75%,19575.750000,2.000000,654.000000,2726.055751,1.718288e+06,2914.911980,1441.137009,1515.102267,2029.574271,31.000000,29.000000,9.00000,3988.033894,4122.115939,26.000000,0.550092,1.157895,1.502425
max,23532.000000,2.000000,24503.000000,3798.837209,6.640940e+07,3488.068182,4603.125000,4467.647059,3236.526948,76.000000,63.000000,25.00000,5852.631579,6110.526316,971.000000,5.465455,6.000000,6.290909


In [47]:
# Equipment Value Analysis (FTE vs Saved Equipment) - ALL demoES

equipment_analysis_query = """
SELECT 
    r.round_num,
    r.team1_winner,
    pr.team,
    pr.player_id,
    pe.saved_eq_val as saved_equipment_value,
    pe.starting_eq_val as starting_equipment_value,
    pe.money_spent,
    pr.eq_val_fte as equipment_value_fte,
    pr.eq_val_re as equipment_value_re,
    CASE WHEN r.round_num = 1 THEN 'pistol'
         WHEN r.round_num = 16 THEN 'second_half_pistol'
         ELSE 'gun_round' END as round_type,
    LAG(pr.eq_val_re) OVER (PARTITION BY pr.player_id, r.id_demo_exports ORDER BY r.round_num) as prev_equipment_value,
    LAG(r.team1_winner) OVER (PARTITION BY pr.player_id, r.id_demo_exports ORDER BY r.round_num) as prev_round_won
FROM player_round_ed pr
JOIN player_economy_ed pe ON pr.id = pe.player_round_id
JOIN rounds_ed r ON pr.round_id = r.id
-- NO demo LIMIT - ANALYZE ALL demoES
ORDER BY r.id_demo_exports, r.round_num, pr.team, pr.player_id
"""

print("🔄 Loading equipment data for ALL demoes... (this may take a moment)")
equipment_data = pd.read_sql(equipment_analysis_query, connection)

# Calculate fresh vs saved equipment
equipment_data['fresh_equipment'] = equipment_data['starting_equipment_value'] - equipment_data['saved_equipment_value'].fillna(0)

# Calculate equipment saved from previous rounds
equipment_data['equipment_saved'] = equipment_data.apply(
    lambda row: row['prev_equipment_value'] if row['prev_round_won'] == 1 else 0, axis=1
)

# FTE (First Time Equipment) vs Saved equipment statistics - COMPREHENSIVE
fte_stats = equipment_data.groupby('round_type').agg({
    'equipment_value_fte': ['mean', 'std', 'min', 'max', 'count'],
    'equipment_value_re': ['mean', 'std'],
    'fresh_equipment': ['mean', 'std'],
    'equipment_saved': ['mean', 'sum'],
    'money_spent': ['mean', 'std']
}).round(2)

print("=== COMPREHENSIVE EQUIPMENT VALUE ANALYSIS (ALL demoES) ===")
print(f"Total equipment transactions analyzed: {len(equipment_data):,}")
print(f"Equipment data spans {equipment_data.groupby(['player_id'])['round_type'].count().sum():,} player-rounds")
print("\nFTE vs Saved Equipment by Round Type (ALL demoES):")
print(fte_stats)

# Equipment efficiency by round type
equipment_efficiency = equipment_data.groupby('round_type').agg({
    'equipment_value_fte': 'mean',
    'money_spent': 'mean',
}).round(0)
equipment_efficiency['equipment_per_dollar'] = equipment_efficiency['equipment_value_fte'] / equipment_efficiency['money_spent']

print(f"\n=== EQUIPMENT EFFICIENCY BY ROUND TYPE ===")
print(equipment_efficiency)

🔄 Loading equipment data for ALL demoes... (this may take a moment)
=== COMPREHENSIVE EQUIPMENT VALUE ANALYSIS (ALL demoES) ===
Total equipment transactions analyzed: 24,896,913
=== COMPREHENSIVE EQUIPMENT VALUE ANALYSIS (ALL demoES) ===
Total equipment transactions analyzed: 24,896,913
Equipment data spans 24,896,913 player-rounds

FTE vs Saved Equipment by Round Type (ALL demoES):
                   equipment_value_fte                               \
                                  mean      std min   max     count   
round_type                                                            
gun_round                      4317.43  2023.65   0  9000  22918499   
pistol                          840.43   149.22   0  6350    989202   
second_half_pistol              844.01   142.81   0  1950    989212   

                   equipment_value_re          fresh_equipment           \
                                 mean      std            mean      std   
round_type                           

## 2. Team Economic Performance

In [48]:
# Team Economic Analysis - ALL demoES

team_economics_query = """
SELECT 
    r.id_demo_exports,
    r.round_num,
    r.team1_winner,
    r.is_ct_t1,
    SUM(CASE WHEN pr.team = 1 THEN pe.money_spent ELSE 0 END) as team1_spent,
    SUM(CASE WHEN pr.team = 2 THEN pe.money_spent ELSE 0 END) as team2_spent,
    SUM(CASE WHEN pr.team = 1 THEN pr.eq_val_fte ELSE 0 END) as team1_equipment,
    SUM(CASE WHEN pr.team = 2 THEN pr.eq_val_fte ELSE 0 END) as team2_equipment,
    AVG(CASE WHEN pr.team = 1 THEN pe.money_spent ELSE NULL END) as team1_avg_spent,
    AVG(CASE WHEN pr.team = 2 THEN pe.money_spent ELSE NULL END) as team2_avg_spent
FROM rounds_ed r
JOIN player_round_ed pr ON r.id = pr.round_id
JOIN player_economy_ed pe ON pr.id = pe.player_round_id
-- NO demo LIMIT - ANALYZE ALL demoES
GROUP BY r.id_demo_exports, r.round_num, r.team1_winner, r.is_ct_t1
ORDER BY r.id_demo_exports, r.round_num
"""

print("🔄 Loading team economics data for ALL demoes...")
team_data = pd.read_sql(team_economics_query, connection)

# Calculate team economic metrics
team_data['spending_advantage'] = team_data['team1_spent'] - team_data['team2_spent']
team_data['equipment_advantage'] = team_data['team1_equipment'] - team_data['team2_equipment']

# Economic efficiency by team
team_data['team1_efficiency'] = team_data['team1_equipment'] / team_data['team1_spent'].replace(0, np.nan)
team_data['team2_efficiency'] = team_data['team2_equipment'] / team_data['team2_spent'].replace(0, np.nan)

# Win rate by economic advantage - COMPREHENSIVE
def categorize_advantage(val):
    if val > 5000: return 'strong_advantage'
    elif val > 1000: return 'advantage'
    elif val > -1000: return 'even'
    elif val > -5000: return 'disadvantage'
    else: return 'strong_disadvantage'

team_data['spending_category'] = team_data['spending_advantage'].apply(categorize_advantage)
win_by_economy = team_data.groupby('spending_category')['team1_winner'].agg(['count', 'mean']).round(3)

print("=== COMPREHENSIVE TEAM ECONOMIC PERFORMANCE (ALL demoES) ===")
print(f"Total rounds analyzed: {len(team_data):,}")
print(f"Unique demoes: {team_data['id_demo_exports'].nunique():,}")
print(f"Average rounds per demo: {len(team_data) / team_data['id_demo_exports'].nunique():.1f}")
print("\nWin rate by spending advantage (ALL demoES):")
print(win_by_economy)
print(f"\nAverage spending difference: ${team_data['spending_advantage'].mean():.0f}")
print(f"Equipment advantage correlation with wins: {team_data['equipment_advantage'].corr(team_data['team1_winner']):.3f}")

# Additional comprehensive statistics
print(f"\n=== COMPREHENSIVE ECONOMIC STATISTICS ===")
print(f"Total money spent across all rounds: ${team_data['team1_spent'].sum() + team_data['team2_spent'].sum():,.0f}")
print(f"Average team spending per round: ${(team_data['team1_spent'] + team_data['team2_spent']).mean():.0f}")
print(f"Equipment efficiency correlation with wins: {team_data[['team1_efficiency', 'team1_winner']].corr().iloc[0,1]:.3f}")

🔄 Loading team economics data for ALL demoes...
=== COMPREHENSIVE TEAM ECONOMIC PERFORMANCE (ALL demoES) ===
Total rounds analyzed: 2,489,790
Unique demoes: 98,937
Average rounds per demo: 25.2

Win rate by spending advantage (ALL demoES):
                      count   mean
spending_category                 
advantage            232049  0.591
disadvantage         227384  0.444
even                 283506  0.513
strong_advantage     872882  0.657
strong_disadvantage  873969  0.372

Average spending difference: $-23
Equipment advantage correlation with wins: 0.433

=== COMPREHENSIVE ECONOMIC STATISTICS ===
Total money spent across all rounds: $66,146,664,550
Average team spending per round: $26567
Equipment efficiency correlation with wins: 0.020
=== COMPREHENSIVE TEAM ECONOMIC PERFORMANCE (ALL demoES) ===
Total rounds analyzed: 2,489,790
Unique demoes: 98,937
Average rounds per demo: 25.2

Win rate by spending advantage (ALL demoES):
                      count   mean
spending_category 

## 3. Kill/Death/Assist Statistics

In [49]:
# KDA Statistics Analysis - ALL demoES (FINAL ROUND STATS ONLY)

# Get final round statistics for each player in each demo
kda_query = """
SELECT 
    pr.player_id,
    r.id_demo_exports,
    -- Get final round stats by taking max values (cumulative stats)
    MAX(pr.kills) as total_kills,
    MAX(pr.deaths) as total_deaths,
    MAX(pr.assists) as total_assists
FROM player_round_ed pr
JOIN (
    SELECT r1.* 
    FROM rounds_ed r1
    INNER JOIN (
        SELECT id_demo_exports, MAX(round_num) as max_round_num
        FROM rounds_ed 
        GROUP BY id_demo_exports
    ) r2 ON r1.id_demo_exports = r2.id_demo_exports 
       AND r1.round_num = r2.max_round_num
) r ON pr.round_id = r.id
GROUP BY r.id_demo_exports, pr.player_id
"""

print("🔄 Loading KDA data for ALL demoes (final round stats only)...")
kda_data = pd.read_sql(kda_query, connection)

# Calculate KDA metrics based on final stats
kda_data['kd_ratio'] = kda_data['total_kills'] / kda_data['total_deaths'].replace(0, 1)
kda_data['kda_ratio'] = (kda_data['total_kills'] + kda_data['total_assists']) / kda_data['total_deaths'].replace(0, 1)
kda_data['kill_participation'] = (kda_data['total_kills'] + kda_data['total_assists'])


print("=== COMPREHENSIVE KILL/DEATH/ASSIST STATISTICS (ALL demoES - FINAL STATS) ===")
print(f"Players analyzed: {len(kda_data):,}")
print(f"Total kills recorded: {kda_data['total_kills'].sum():,}")
print(f"Total deaths recorded: {kda_data['total_deaths'].sum():,}")
print(f"Total assists recorded: {kda_data['total_assists'].sum():,}")

print("\nTop 10 performers by KD ratio (ALL demoES - FINAL STATS):")
print(kda_data.nlargest(10, 'kd_ratio')[['player_id', 'total_kills', 'total_deaths', 'kd_ratio']])

print(f"\n=== COMPREHENSIVE PERFORMANCE STATISTICS ===")
print(f"Average KD ratio: {kda_data['kd_ratio'].mean():.3f}")
print(f"Median KD ratio: {kda_data['kd_ratio'].median():.3f}")



🔄 Loading KDA data for ALL demoes (final round stats only)...
=== COMPREHENSIVE KILL/DEATH/ASSIST STATISTICS (ALL demoES - FINAL STATS) ===
Players analyzed: 989,202
Total kills recorded: 17,297,729.0
Total deaths recorded: 17,357,506.0
Total assists recorded: 3,451,995.0

Top 10 performers by KD ratio (ALL demoES - FINAL STATS):
        player_id  total_kills  total_deaths  kd_ratio
866561       8084         26.0           1.0      26.0
967573      13058         26.0           0.0      26.0
56590        9056         24.0           1.0      24.0
151737      11495         24.0           1.0      24.0
442397       8766         24.0           1.0      24.0
267055      10870         23.0           1.0      23.0
808949       8084         23.0           1.0      23.0
54807       12573         22.0           1.0      22.0
227946      11139         22.0           1.0      22.0
525596       8575         21.0           1.0      21.0

=== COMPREHENSIVE PERFORMANCE STATISTICS ===
Average KD ratio:

## 4. Weapon Analysis & Efficiency

In [50]:
# Weapon Usage and Efficiency Analysis - ALL demoES

# Weapon purchase patterns from inventory - ALL demoES
weapon_purchases_query = """
SELECT 
    i.equipment_string,
    i.equipment_class,
    COUNT(*) as purchase_count,
    AVG(pe.money_spent) as avg_round_spending,
    COUNT(DISTINCT r.id) as rounds_featured
FROM inventory_round_ed i
JOIN player_economy_ed pe ON i.player_round_id = pe.player_round_id
JOIN rounds_ed r ON i.round_id = r.id
-- NO demo LIMIT - ANALYZE ALL demoES
WHERE i.event_type = 1  -- Purchase events
group by 1,2
ORDER BY purchase_count DESC
"""

print("🔄 Loading weapon purchase data for ALL demoes...")
weapon_purchases = pd.read_sql(weapon_purchases_query, connection)

# Weapon kill efficiency - ALL demoES
weapon_efficiency_query = """
SELECT 
    k.weapon,
    COUNT(*) as total_kills,
    COUNT(DISTINCT r.id) as rounds_used
FROM kills_round_ed k
JOIN rounds_ed r ON k.round_id = r.id
-- NO demo LIMIT - ANALYZE ALL demoES
GROUP BY k.weapon
ORDER BY total_kills DESC
"""

print("🔄 Loading weapon efficiency data for ALL demoes...")
weapon_efficiency = pd.read_sql(weapon_efficiency_query, connection)

# Calculate weapon effectiveness metrics
weapon_efficiency['kills_per_round'] = weapon_efficiency['total_kills'] / weapon_purchases['rounds_featured']
weapon_efficiency['kills_per_buy'] = weapon_efficiency['total_kills'] / weapon_purchases['purchase_count']

# Most effective weapons
top_weapons = weapon_efficiency.nlargest(15, 'total_kills')

print("=== COMPREHENSIVE WEAPON ANALYSIS (ALL demoES) ===")
print(f"Total weapon purchases analyzed: {weapon_purchases['purchase_count'].sum():,}")
print(f"Unique weapons purchased: {len(weapon_purchases):,}")
print(f"Total weapon kills analyzed: {weapon_efficiency['total_kills'].sum():,}")
print(f"Unique weapons used for kills: {len(weapon_efficiency):,}")

print("\nMost purchased weapons (ALL demoES):")
print(weapon_purchases.head(15)[['equipment_string', 'purchase_count', 'rounds_featured']])

print("\nMost effective weapons by total kills (ALL demoES):")
print(top_weapons[['weapon', 'total_kills',  'kills_per_round']])

print(f"\n=== COMPREHENSIVE WEAPON STATISTICS ===")
print(f"Most deadly weapon: {weapon_efficiency.loc[weapon_efficiency['total_kills'].idxmax(), 'weapon']}")
print(f"Average kills per weapon per round: {weapon_efficiency['kills_per_round'].mean():.3f}")


🔄 Loading weapon purchase data for ALL demoes...
🔄 Loading weapon efficiency data for ALL demoes...
🔄 Loading weapon efficiency data for ALL demoes...
=== COMPREHENSIVE WEAPON ANALYSIS (ALL demoES) ===
Total weapon purchases analyzed: 127,059,754
Unique weapons purchased: 43
Total weapon kills analyzed: 17,584,321
Unique weapons used for kills: 1

Most purchased weapons (ALL demoES):
      equipment_string  purchase_count  rounds_featured
0                Knife        24927149          2489790
1        Smoke Grenade        17934683          2473659
2            Flashbang        16753819          2464052
3             Glock-18         9195369          2388287
4                USP-S         8955995          2421897
5           HE Grenade         8575983          2280262
6                AK-47         7618687          2092239
7              Molotov         7278050          1971548
8   Incendiary Grenade         6354727          1787989
9                 M4A4         2655842          12922

## 5. Inventory Composition Analysis

In [51]:
# General Inventory Composition Analysis - ALL demoES (EQUIPMENT USAGE FOCUS)

inventory_composition_query = """
SELECT 
    i.equipment_class,
    i.equipment_string,
    COUNT(*) as usage_count,
    COUNT(DISTINCT pr.player_id) as unique_players,
    COUNT(DISTINCT r.id) as rounds_involved,
    COUNT(DISTINCT r.id_demo_exports) as demos_involved
FROM inventory_round_ed i
JOIN player_round_ed pr ON i.player_round_id = pr.id
JOIN rounds_ed r ON i.round_id = r.id
-- NO demo LIMIT - ANALYZE ALL demoES
WHERE i.event_type = 1  -- Only FTE
  AND i.equipment_class != 5  -- Exclude knives
GROUP BY i.equipment_class, i.equipment_string
HAVING COUNT(*) >= 10  -- Minimum 10 usage events for meaningful statistics
ORDER BY usage_count DESC
"""

print("🔄 Loading inventory usage data for ALL demoes (equipment usage only)...")
inventory_data = pd.read_sql(inventory_composition_query, connection)

# Equipment class breakdown
equipment_classes = {
    1: 'Pistols', 
    2: 'SMGs',
    3: 'Shotguns',
    4: 'Rifles',
    5: 'Basics',
    6: 'Grenades'
}

inventory_data['class_name'] = inventory_data['equipment_class'].map(equipment_classes).fillna('Other')

# Equipment usage by class - COMPREHENSIVE
class_summary = inventory_data.groupby('class_name').agg({
    'usage_count': 'sum',
    'unique_players': 'mean',
    'rounds_involved': 'mean',
    'demos_involved': 'mean'
}).round(2)

# Most commonly used equipment across rounds
top_equipment = inventory_data.nlargest(25, 'usage_count')[['equipment_string', 'class_name', 'usage_count', 'rounds_involved', 'demos_involved']]

# Equipment usage distribution analysis
usage_distribution = inventory_data.groupby('class_name').agg({
    'usage_count': ['sum', 'mean', 'std', 'min', 'max'],
    'rounds_involved': ['sum', 'mean'],
    'demos_involved': ['mean']
}).round(2)

# Round penetration analysis - how many rounds each equipment appears in
round_penetration = inventory_data.copy()
round_penetration['round_penetration_rate'] = round_penetration['rounds_involved'] / round_penetration['demos_involved']

# Most common inventory compositions per round
inventory_composition_query_detailed = """
WITH round_inventory AS (
    SELECT 
        r.round_id,
        r.id_demo_exports,
        pr.player_id,
        STRING_AGG(i.equipment_string, ', ' ORDER BY i.equipment_string) as inventory_composition
    FROM inventory_round_ed i
    JOIN player_round_ed pr ON i.player_round_id = pr.id
    JOIN rounds_ed r ON i.round_id = r.id
    WHERE i.event_type = 1  -- Only pickup/usage events
      AND i.equipment_class != 5 -- Exclude knives
    GROUP BY r.round_id, r.id_demo_exports, pr.player_id
    HAVING COUNT(*) >= 2  -- At least 2 pieces of equipment
)
SELECT 
    inventory_composition,
    COUNT(*) as composition_frequency,
    COUNT(DISTINCT player_id) as unique_players_with_composition,
    COUNT(DISTINCT id_demo_exports) as demoes_featured
FROM round_inventory
GROUP BY inventory_composition
HAVING COUNT(*) >= 5  -- Minimum 5 occurrences
ORDER BY composition_frequency DESC
LIMIT 20
"""

print("🔄 Loading detailed inventory composition patterns...")
composition_patterns = pd.read_sql(inventory_composition_query_detailed, connection)

print("=== COMPREHENSIVE EQUIPMENT USAGE ANALYSIS (ALL demoES) ===")
print(f"Total equipment usage events analyzed: {inventory_data['usage_count'].sum():,}")
print(f"Unique equipment items tracked: {len(inventory_data):,}")
print(f"Total rounds with equipment usage: {inventory_data['rounds_involved'].sum():,}")
print(f"Total demoes covered: {inventory_data['demoes_involved'].max():,}")

print("\nEquipment usage by class (ALL demoES):")
print(class_summary)

print(f"\n=== EQUIPMENT USAGE DISTRIBUTION ===")
print(usage_distribution)

print(f"\n=== MOST COMMONLY USED EQUIPMENT (TOP 25) ===")
print(top_equipment)

print(f"\n=== ROUND PENETRATION ANALYSIS ===")
penetration_stats = round_penetration.nlargest(15, 'rounds_involved')[['equipment_string', 'class_name', 'rounds_involved', 'demoes_involved', 'round_penetration_rate']]
print("Equipment with highest round penetration:")
print(penetration_stats)

print(f"\n=== MOST COMMON INVENTORY COMPOSITIONS ===")
print("Top inventory combinations per player per round:")
print(composition_patterns)

# Statistical insights
print(f"\n=== COMPREHENSIVE USAGE STATISTICS ===")
print(f"Most used equipment: {top_equipment.iloc[0]['equipment_string']} ({top_equipment.iloc[0]['usage_count']:,} uses)")
print(f"Most versatile equipment class: {class_summary.loc[class_summary['usage_count'].idxmax()].name}")
print(f"Average equipment uses per item: {inventory_data['usage_count'].mean():.1f}")
print(f"Equipment diversity: {len(inventory_data):,} unique equipment items")

# Usage frequency categories
print(f"\n=== USAGE FREQUENCY CATEGORIES ===")
print(f"High usage (1000+ uses): {(inventory_data['usage_count'] >= 1000).sum()}")
print(f"Medium usage (100-999 uses): {((inventory_data['usage_count'] >= 100) & (inventory_data['usage_count'] < 1000)).sum()}")
print(f"Low usage (10-99 uses): {((inventory_data['usage_count'] >= 10) & (inventory_data['usage_count'] < 100)).sum()}")

# Class distribution
class_distribution = inventory_data.groupby('class_name')['usage_count'].sum().sort_values(ascending=False)
print(f"\n=== EQUIPMENT CLASS USAGE DISTRIBUTION ===")
for class_name, usage in class_distribution.items():
    percentage = (usage / inventory_data['usage_count'].sum()) * 100
    print(f"{class_name}: {usage:,} uses ({percentage:.1f}%)")

🔄 Loading inventory usage data for ALL demoes (equipment usage only)...
🔄 Loading detailed inventory composition patterns...
🔄 Loading detailed inventory composition patterns...


DatabaseError: Execution failed on sql '
WITH round_inventory AS (
    SELECT 
        r.round_id,
        r.id_demo_exports,
        pr.player_id,
        STRING_AGG(i.equipment_string, ', ' ORDER BY i.equipment_string) as inventory_composition
    FROM inventory_round_ed i
    JOIN player_round_ed pr ON i.player_round_id = pr.id
    JOIN rounds_ed r ON i.round_id = r.id
    WHERE i.event_type = 1  -- Only pickup/usage events
      AND i.equipment_class != 5 -- Exclude knives
    GROUP BY r.round_id, r.id_demo_exports, pr.player_id
    HAVING COUNT(*) >= 2  -- At least 2 pieces of equipment
)
SELECT 
    inventory_composition,
    COUNT(*) as composition_frequency,
    COUNT(DISTINCT player_id) as unique_players_with_composition,
    COUNT(DISTINCT id_demo_exports) as demoes_featured
FROM round_inventory
GROUP BY inventory_composition
HAVING COUNT(*) >= 5  -- Minimum 5 occurrences
ORDER BY composition_frequency DESC
LIMIT 20
': column r.round_id does not exist
LINE 4:         r.round_

## 6. Win/Loss Streaks and Patterns

In [ ]:
# Win/Loss Streaks and Pattern Analysis - ALL demoES (Using Pre-calculated Streak Data)

# Load pre-calculated streak data from the streak table
streak_query = """
SELECT 
    s.id,
    s.round_id,
    s.round_num,
    s.t1_win_streak,
    s.t1_loss_streak,
    s.t1_win_streak_cons,
    s.t1_loss_streak_cons,
    s.t1_wins_last_1,
    s.t1_wins_last_2,
    s.t1_wins_last_3,
    s.t1_wins_last_4,
    s.t1_wins_last_5,
    s.t1_wins_last_6,
    s.t1_wins_last_7,
    s.t2_win_streak,
    s.t2_loss_streak,
    s.t2_win_streak_cons,
    s.t2_loss_streak_cons,
    s.t2_wins_last_1,
    s.t2_wins_last_2,
    s.t2_wins_last_3,
    s.t2_wins_last_4,
    s.t2_wins_last_5,
    s.t2_wins_last_6,
    s.t2_wins_last_7,
    r.id_demo_exports,
    r.team1_winner
FROM streaks s
JOIN rounds_ed r ON s.round_id = r.id
-- NO demo LIMIT - ANALYZE ALL demoES
ORDER BY r.id_demo_exports, s.round_num
"""

print("🔄 Loading pre-calculated streak data for ALL demoes...")
streak_data = pd.read_sql(streak_query, connection)

# Calculate current streak lengths (both teams)
streak_data['t1_current_streak'] = np.where(
    streak_data['t1_win_streak'] > 0, 
    streak_data['t1_win_streak'], 
    -streak_data['t1_loss_streak']
)
streak_data['t2_current_streak'] = np.where(
    streak_data['t2_win_streak'] > 0, 
    streak_data['t2_win_streak'], 
    -streak_data['t2_loss_streak']
)

# Comprehensive streak statistics
print("=== COMPREHENSIVE WIN/LOSS STREAKS ANALYSIS (ALL demoES) ===")
print(f"Total rounds analyzed: {len(streak_data):,}")
print(f"Total demos analyzed: {streak_data['id_demo_exports'].nunique():,}")

# Current streak distribution analysis
print(f"\n=== CURRENT STREAK STATISTICS ===")
print(f"Team 1 - Max win streak: {streak_data['t1_win_streak'].max()}")
print(f"Team 1 - Max loss streak: {streak_data['t1_loss_streak'].max()}")
print(f"Team 1 - Average win streak: {streak_data['t1_win_streak'].mean():.2f}")
print(f"Team 1 - Average loss streak: {streak_data['t1_loss_streak'].mean():.2f}")

print(f"\nTeam 2 - Max win streak: {streak_data['t2_win_streak'].max()}")
print(f"Team 2 - Max loss streak: {streak_data['t2_loss_streak'].max()}")
print(f"Team 2 - Average win streak: {streak_data['t2_win_streak'].mean():.2f}")
print(f"Team 2 - Average loss streak: {streak_data['t2_loss_streak'].mean():.2f}")

# Consecutive streak analysis
print(f"\n=== CONSECUTIVE STREAK STATISTICS ===")
print(f"Team 1 - Max consecutive wins: {streak_data['t1_win_streak_cons'].max()}")
print(f"Team 1 - Max consecutive losses: {streak_data['t1_loss_streak_cons'].max()}")
print(f"Team 1 - Average consecutive wins: {streak_data['t1_win_streak_cons'].mean():.2f}")
print(f"Team 1 - Average consecutive losses: {streak_data['t1_loss_streak_cons'].mean():.2f}")

print(f"\nTeam 2 - Max consecutive wins: {streak_data['t2_win_streak_cons'].max()}")
print(f"Team 2 - Max consecutive losses: {streak_data['t2_loss_streak_cons'].max()}")
print(f"Team 2 - Average consecutive wins: {streak_data['t2_win_streak_cons'].mean():.2f}")
print(f"Team 2 - Average consecutive losses: {streak_data['t2_loss_streak_cons'].mean():.2f}")

# Momentum analysis using rolling window data
momentum_columns = ['t1_wins_last_1', 't1_wins_last_2', 't1_wins_last_3', 't1_wins_last_4', 't1_wins_last_5', 't1_wins_last_6', 't1_wins_last_7']
momentum_analysis = {}

for i, col in enumerate(momentum_columns, 1):
    momentum_stats = streak_data.groupby(col)['team1_winner'].agg(['count', 'mean']).round(3)
    momentum_analysis[f'last_{i}_rounds'] = momentum_stats
    print(f"\n=== MOMENTUM ANALYSIS - LAST {i} ROUND(S) ===")
    print(f"Win rate by wins in last {i} round(s):")
    print(momentum_stats)

# Streak momentum patterns
print(f"\n=== STREAK MOMENTUM PATTERNS ===")

# High momentum scenarios (multiple wins in recent rounds)
high_momentum_t1 = streak_data[
    (streak_data['t1_wins_last_3'] >= 2) & 
    (streak_data['t1_wins_last_5'] >= 3)
]
print(f"Team 1 high momentum rounds (2+ wins in last 3, 3+ in last 5): {len(high_momentum_t1):,}")
if len(high_momentum_t1) > 0:
    print(f"Team 1 win rate during high momentum: {high_momentum_t1['team1_winner'].mean():.1%}")

# Low momentum scenarios (multiple losses)
low_momentum_t1 = streak_data[
    (streak_data['t1_wins_last_3'] <= 1) & 
    (streak_data['t1_wins_last_5'] <= 1)
]
print(f"Team 1 low momentum rounds (≤1 win in last 3, ≤1 in last 5): {len(low_momentum_t1):,}")
if len(low_momentum_t1) > 0:
    print(f"Team 1 win rate during low momentum: {low_momentum_t1['team1_winner'].mean():.1%}")

# Comeback analysis - win after losing streaks
comeback_analysis = streak_data[streak_data['t1_loss_streak'] >= 3]
print(f"\n=== COMEBACK ANALYSIS ===")
print(f"Rounds where Team 1 had 3+ loss streak: {len(comeback_analysis):,}")
if len(comeback_analysis) > 0:
    print(f"Team 1 comeback rate (win after 3+ losses): {comeback_analysis['team1_winner'].mean():.1%}")

comeback_analysis_major = streak_data[streak_data['t1_loss_streak'] >= 5]
print(f"Rounds where Team 1 had 5+ loss streak: {len(comeback_analysis_major):,}")
if len(comeback_analysis_major) > 0:
    print(f"Team 1 major comeback rate (win after 5+ losses): {comeback_analysis_major['team1_winner'].mean():.1%}")

# Streak distribution analysis
print(f"\n=== STREAK LENGTH DISTRIBUTION ===")
win_streak_dist = streak_data['t1_win_streak'].value_counts().sort_index()
loss_streak_dist = streak_data['t1_loss_streak'].value_counts().sort_index()

print("Team 1 Win Streak Distribution (top 10):")
print(win_streak_dist.head(10))
print("\nTeam 1 Loss Streak Distribution (top 10):")
print(loss_streak_dist.head(10))

In [ ]:
# Advanced Streak Pattern Analysis

# Streak transition analysis - how often do teams break opponent streaks
print("=== ADVANCED STREAK PATTERN ANALYSIS ===")

# Streak break analysis
def analyze_streak_breaks(data):
    """Analyze how often teams break each other's streaks"""
    
    # Team 1 breaking Team 2's streaks
    t2_streak_breaks = data[
        (data['t2_win_streak'] >= 3) & 
        (data['team1_winner'] == True)
    ]
    
    # Team 2 breaking Team 1's streaks  
    t1_streak_breaks = data[
        (data['t1_win_streak'] >= 3) & 
        (data['team1_winner'] == False)
    ]
    
    return t1_streak_breaks, t2_streak_breaks

t1_breaks, t2_breaks = analyze_streak_breaks(streak_data)

print(f"Team 1 streak breaks (T1 had 3+ wins, T2 won): {len(t1_breaks):,}")
print(f"Team 2 streak breaks (T2 had 3+ wins, T1 won): {len(t2_breaks):,}")

# Streak sustainability analysis
sustainability_analysis = {}
for streak_length in [3, 4, 5, 6, 7]:
    # How often does a team continue their streak vs get broken
    t1_with_streak = streak_data[streak_data['t1_win_streak'] >= streak_length]
    if len(t1_with_streak) > 0:
        continuation_rate = t1_with_streak['team1_winner'].mean()
        sustainability_analysis[f't1_{streak_length}_rounds'] = {
            'rounds': len(t1_with_streak),
            'continuation_rate': continuation_rate
        }
    
    t2_with_streak = streak_data[streak_data['t2_win_streak'] >= streak_length]
    if len(t2_with_streak) > 0:
        # For team 2, continuation means team1_winner = False
        continuation_rate = 1 - t2_with_streak['team1_winner'].mean()
        sustainability_analysis[f't2_{streak_length}_rounds'] = {
            'rounds': len(t2_with_streak),
            'continuation_rate': continuation_rate
        }

print(f"\n=== STREAK SUSTAINABILITY ANALYSIS ===")
for key, stats in sustainability_analysis.items():
    print(f"{key}: {stats['rounds']:,} rounds, {stats['continuation_rate']:.1%} continuation rate")

# Rolling momentum correlation analysis
momentum_correlations = {}
for i in range(1, 8):
    col_t1 = f't1_wins_last_{i}'
    col_t2 = f't2_wins_last_{i}'
    
    if col_t1 in streak_data.columns:
        corr_t1 = streak_data[col_t1].corr(streak_data['team1_winner'])
        momentum_correlations[f'T1_last_{i}'] = corr_t1
    
    if col_t2 in streak_data.columns:
        # For team 2, we need negative correlation (their wins = team1_winner = False)
        corr_t2 = streak_data[col_t2].corr(1 - streak_data['team1_winner'].astype(int))
        momentum_correlations[f'T2_last_{i}'] = corr_t2

print(f"\n=== MOMENTUM CORRELATION ANALYSIS ===")
print("Correlation between recent wins and continuing to win:")
for period, correlation in momentum_correlations.items():
    print(f"{period}: {correlation:.3f}")

# Momentum shift detection
momentum_shifts = []
for id_demo_exports in streak_data['id_demo_exports'].unique()[:50]:  # Sample analysis on first 50 demoes
    demo_data = streak_data[streak_data['id_demo_exports'] == id_demo_exports].sort_values('round_num')
    
    if len(demo_data) < 10:  # Skip very short demoes
        continue
        
    # Look for momentum shifts (team losing streak to winning streak)
    for i in range(1, len(demo_data)):
        prev_round = demo_data.iloc[i-1]
        curr_round = demo_data.iloc[i]
        
        # Team 1 momentum shift (from losing to winning)
        if (prev_round['t1_loss_streak'] >= 2 and 
            curr_round['t1_win_streak'] >= 1):
            momentum_shifts.append({
                'id_demo_exports': id_demo_exports,
                'round_num': curr_round['round_num'],
                'team': 1,
                'shift_type': 'loss_to_win',
                'prev_streak': prev_round['t1_loss_streak']
            })
        
        # Team 2 momentum shift  
        if (prev_round['t2_loss_streak'] >= 2 and 
            curr_round['t2_win_streak'] >= 1):
            momentum_shifts.append({
                'id_demo_exports': id_demo_exports,
                'round_num': curr_round['round_num'],
                'team': 2,
                'shift_type': 'loss_to_win',
                'prev_streak': prev_round['t2_loss_streak']
            })

momentum_shifts_df = pd.DataFrame(momentum_shifts)

print(f"\n=== MOMENTUM SHIFT ANALYSIS ===")
if len(momentum_shifts_df) > 0:
    print(f"Total momentum shifts detected: {len(momentum_shifts_df):,}")
    print(f"Average previous losing streak before shift: {momentum_shifts_df['prev_streak'].mean():.1f}")
    print(f"Team 1 momentum shifts: {len(momentum_shifts_df[momentum_shifts_df['team'] == 1]):,}")
    print(f"Team 2 momentum shifts: {len(momentum_shifts_df[momentum_shifts_df['team'] == 2]):,}")
    
    # Momentum shift distribution by previous streak length
    shift_dist = momentum_shifts_df.groupby('prev_streak').size()
    print(f"\nMomentum shifts by previous losing streak length:")
    print(shift_dist.head(10))
else:
    print("No significant momentum shifts detected in sample")

# Psychological pressure analysis
pressure_scenarios = streak_data[
    (streak_data['t1_loss_streak'] >= 4) | 
    (streak_data['t2_loss_streak'] >= 4)
]

print(f"\n=== PSYCHOLOGICAL PRESSURE ANALYSIS ===")
print(f"High pressure rounds (4+ loss streak): {len(pressure_scenarios):,}")

if len(pressure_scenarios) > 0:
    # Performance under pressure
    t1_under_pressure = pressure_scenarios[pressure_scenarios['t1_loss_streak'] >= 4]
    t2_under_pressure = pressure_scenarios[pressure_scenarios['t2_loss_streak'] >= 4]
    
    if len(t1_under_pressure) > 0:
        t1_pressure_performance = t1_under_pressure['team1_winner'].mean()
        print(f"Team 1 performance under pressure (4+ losses): {t1_pressure_performance:.1%}")
    
    if len(t2_under_pressure) > 0:
        t2_pressure_performance = 1 - t2_under_pressure['team1_winner'].mean()
        print(f"Team 2 performance under pressure (4+ losses): {t2_pressure_performance:.1%}")

# Streak pattern summary
print(f"\n=== STREAK PATTERN SUMMARY ===")
print(f"Most common win streak length: {streak_data['t1_win_streak'].mode().iloc[0] if len(streak_data['t1_win_streak'].mode()) > 0 else 'N/A'}")
print(f"Most common loss streak length: {streak_data['t1_loss_streak'].mode().iloc[0] if len(streak_data['t1_loss_streak'].mode()) > 0 else 'N/A'}")
print(f"Percentage of rounds with active streaks (3+): {((streak_data['t1_win_streak'] >= 3) | (streak_data['t1_loss_streak'] >= 3) | (streak_data['t2_win_streak'] >= 3) | (streak_data['t2_loss_streak'] >= 3)).mean():.1%}")

In [ ]:
# Ways to Win Analysis

round_end_reasons = {
    1: 'Target eliminated',
    2: 'VIP escaped', 
    3: 'VIP assassinated',
    4: 'Terrorists escaped',
    5: 'CTs prevented escape',
    6: 'Escaping terrorists neutralized',
    7: 'Bomb defused',
    8: 'Bomb exploded',
    9: 'Terrorists win',
    10: 'CTs win',
    11: 'Draw',
    12: 'Hostage rescued',
    13: 'Hostage not rescued'
}

ways_to_win_query = """
SELECT 
    r.round_end_reason,
    r.ct_winner,
    COUNT(*) as frequency,
    AVG(CASE WHEN r.team1_winner THEN 1.0 ELSE 0.0 END) as team1_win_rate
FROM rounds_ed r
WHERE r.id_demo_exports IN (SELECT DISTINCT id_demo_exports FROM rounds_ed LIMIT 10)
  AND r.round_end_reason IS NOT NULL
GROUP BY r.round_end_reason, r.ct_winner
ORDER BY frequency DESC
"""

ways_data = pd.read_sql(ways_to_win_query, connection)
ways_data['reason_name'] = ways_data['round_end_reason'].map(round_end_reasons).fillna('Unknown')

print("=== WAYS TO WIN ANALYSIS ===")
print("Round end reasons frequency:")
print(ways_data[['reason_name', 'ct_winner', 'frequency', 'team1_win_rate']])

# CT vs T win methods
ct_wins = ways_data[ways_data['ct_winner'] == True]
t_wins = ways_data[ways_data['ct_winner'] == False]

print(f"\nCT win methods (top 3):")
print(ct_wins.nlargest(3, 'frequency')[['reason_name', 'frequency']])
print(f"\nT win methods (top 3):")
print(t_wins.nlargest(3, 'frequency')[['reason_name', 'frequency']])

## 7. Bomb Plant/Defuse Outcomes

In [ ]:
# Bomb Plant/Defuse Analysis - ALL demoES

bomb_events_query = """
SELECT 
    r.id_demo_exports,
    r.round_num,
    r.team1_winner,
    r.ct_winner,
    b.bomb_event_type,
    b.site,
    b.carrier_hltv_id,
    COUNT(*) OVER (PARTITION BY r.id) as events_in_round
FROM bomb_events_round_ed b
JOIN rounds_ed r ON b.round_id = r.id
-- NO demo LIMIT - ANALYZE ALL demoES
ORDER BY r.id_demo_exports, r.round_num, b.tick
"""

print("🔄 Loading bomb events data for ALL demoes...")
bomb_data = pd.read_sql(bomb_events_query, connection)

bomb_event_types = {
    1: 'Plant begin',
    2: 'Plant abort', 
    3: 'Plant complete',
    4: 'Defuse begin',
    5: 'Defuse abort',
    6: 'Defuse complete',
    7: 'Explode'
}

bomb_data['event_name'] = bomb_data['bomb_event_type'].map(bomb_event_types).fillna('Unknown')

# Plant success analysis - COMPREHENSIVE
plants = bomb_data[bomb_data['bomb_event_type'] == 3]  # Plant complete
defuses = bomb_data[bomb_data['bomb_event_type'] == 6]  # Defuse complete
explosions = bomb_data[bomb_data['bomb_event_type'] == 7]  # Explode
plant_begins = bomb_data[bomb_data['bomb_event_type'] == 1]  # Plant begin
defuse_begins = bomb_data[bomb_data['bomb_event_type'] == 4]  # Defuse begin

# Site analysis - COMPREHENSIVE
site_analysis = plants.groupby('site').agg({
    'round_num': 'count',
    'ct_winner': 'mean',  # CT win rate when bomb planted on this site
    'id_demo_exports': 'nunique'
}).rename(columns={'round_num': 'plants', 'ct_winner': 'ct_win_rate_after_plant', 'id_demo_exports': 'demoes_featured'})

# Bomb round outcomes
bomb_rounds = bomb_data['round_num'].unique()
bomb_round_data = bomb_data.groupby(['id_demo_exports', 'round_num']).agg({
    'team1_winner': 'first',
    'ct_winner': 'first',
    'bomb_event_type': lambda x: list(x)
}).reset_index()

# Success rates - COMPREHENSIVE
plant_attempts = bomb_data[bomb_data['bomb_event_type'].isin([1, 2, 3])]
successful_plants = len(plants)
total_plant_attempts = len(plant_attempts.groupby(['id_demo_exports', 'round_num']))

defuse_attempts = bomb_data[bomb_data['bomb_event_type'].isin([4, 5, 6])]
successful_defuses = len(defuses)
total_defuse_attempts = len(defuse_attempts.groupby(['id_demo_exports', 'round_num']))

print("=== COMPREHENSIVE BOMB PLANT/DEFUSE ANALYSIS (ALL demoES) ===")
print(f"Total bomb events analyzed: {len(bomb_data):,}")
print(f"demoes with bomb events: {bomb_data['id_demo_exports'].nunique():,}")
print(f"Rounds with bomb events: {len(bomb_rounds):,}")

print(f"\nTotal bomb plants: {successful_plants:,}")
print(f"Total defuses: {successful_defuses:,}")
print(f"Total explosions: {len(explosions):,}")
print(f"Total plant attempts: {total_plant_attempts:,}")
print(f"Total defuse attempts: {total_defuse_attempts:,}")

if total_plant_attempts > 0:
    print(f"Plant success rate: {(successful_plants/total_plant_attempts)*100:.1f}%")
else:
    print("No plant attempts found")
    
if total_defuse_attempts > 0:
    print(f"Defuse success rate: {(successful_defuses/total_defuse_attempts)*100:.1f}%")
else:
    print("No defuse attempts found")

print("\nBomb site analysis (ALL demoES):")
print(site_analysis)

# Round outcomes when bomb is planted - COMPREHENSIVE
if successful_plants > 0:
    plant_round_outcomes = bomb_data[bomb_data['round_num'].isin(plants['round_num'])]
    ct_win_after_plant = plant_round_outcomes['ct_winner'].mean()
    print(f"\nCT win rate after bomb plant: {ct_win_after_plant:.1%}")

# Event type distribution
event_distribution = bomb_data.groupby('event_name').agg({
    'id_demo_exports': 'nunique',
    'round_num': 'count'
}).rename(columns={'id_demo_exports': 'demoes', 'round_num': 'occurrences'})

print(f"\n=== COMPREHENSIVE BOMB EVENT DISTRIBUTION ===")
print(event_distribution)

# Bomb outcome analysis
bomb_outcomes = {
    'Successful plants': successful_plants,
    'Successful defuses': successful_defuses,
    'Explosions': len(explosions),
    'Aborted plants': len(bomb_data[bomb_data['bomb_event_type'] == 2]),
    'Aborted defuses': len(bomb_data[bomb_data['bomb_event_type'] == 5])
}

print(f"\n=== BOMB EVENT OUTCOMES SUMMARY ===")
for outcome, count in bomb_outcomes.items():
    print(f"{outcome}: {count:,}")

## 8. Regression Analysis on Round Outcomes

In [ ]:
# Regression Analysis on Round Outcomes - ALL demoES

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler

# Comprehensive dataset for regression analysis - ALL demoES
regression_query = """
SELECT 
    r.id as round_id,
    r.id_demo_exports,
    r.round_num,
    r.team1_winner,
    r.is_ct_t1,
    -- Economic features
    SUM(CASE WHEN pr.team = 1 THEN pe.money_spent ELSE 0 END) as team1_spent,
    SUM(CASE WHEN pr.team = 2 THEN pe.money_spent ELSE 0 END) as team2_spent,
    SUM(CASE WHEN pr.team = 1 THEN pr.eq_val_fte ELSE 0 END) as team1_equipment,
    SUM(CASE WHEN pr.team = 2 THEN pr.eq_val_fte ELSE 0 END) as team2_equipment,
    -- Performance features
    SUM(CASE WHEN pr.team = 1 THEN pr.kills ELSE 0 END) as team1_kills,
    SUM(CASE WHEN pr.team = 2 THEN pr.kills ELSE 0 END) as team2_kills,
FROM rounds_ed r
JOIN player_round_ed pr ON r.id = pr.round_id
JOIN player_economy_ed pe ON pr.id = pe.player_round_id
-- NO demo LIMIT - ANALYZE ALL demoES
GROUP BY r.id, r.id_demo_exports, r.round_num, r.team1_winner, r.is_ct_t1
ORDER BY r.id_demo_exports, r.round_num
"""

print("🔄 Loading regression dataset for ALL demoes... (this may take several moments)")
regression_data = pd.read_sql(regression_query, connection)

# Feature engineering
regression_data['spending_advantage'] = regression_data['team1_spent'] - regression_data['team2_spent']
regression_data['equipment_advantage'] = regression_data['team1_equipment'] - regression_data['team2_equipment']
regression_data['kill_advantage'] = regression_data['team1_kills'] - regression_data['team2_kills']

# Lag features (previous round performance)
regression_data = regression_data.sort_values(['id_demo_exports', 'round_num'])
regression_data['prev_team1_winner'] = regression_data.groupby('id_demo_exports')['team1_winner'].shift(1)

# Round type features
regression_data['is_pistol_round'] = ((regression_data['round_num'] == 1) | (regression_data['round_num'] == 16)).astype(int)
regression_data['is_eco_round'] = (regression_data['team1_spent'] + regression_data['team2_spent'] < 10000).astype(int)

# Prepare features for modeling
feature_columns = [
    'spending_advantage', 'equipment_advantage',
    'kill_advantage', 'is_ct_t1', 'round_num',
    'is_pistol_round', 'is_eco_round', 'prev_team1_winner', 'prev_money_advantage'
]

# Remove rows with missing values
model_data = regression_data[feature_columns + ['team1_winner']].dropna()

X = model_data[feature_columns]
y = model_data['team1_winner'].astype(int)

# Train-test split with stratification for better representation
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("=== COMPREHENSIVE REGRESSION ANALYSIS SETUP (ALL demoES) ===")
print(f"Total rounds for analysis: {len(model_data):,}")
print(f"Total demoes included: {regression_data['id_demo_exports'].nunique():,}")
print(f"Features: {len(feature_columns)}")
print(f"Training set: {len(X_train):,} rounds")
print(f"Test set: {len(X_test):,} rounds")
print(f"Class balance - Team 1 wins: {y.mean():.1%}")
print(f"Features: {feature_columns}")

# Data quality checks
print(f"\n=== DATA QUALITY SUMMARY ===")
print(f"Missing values removed: {len(regression_data) - len(model_data):,} rounds")
print(f"Data completeness: {len(model_data)/len(regression_data):.1%}")
print(f"Average rounds per demo: {len(model_data)/regression_data['id_demo_exports'].nunique():.1f}")

In [ ]:
# Multiple Regression Models

# 1. Logistic Regression
logistic_model = LogisticRegression(random_state=42, max_iter=1000)
logistic_model.fit(X_train_scaled, y_train)

# Predictions
y_pred_logistic = logistic_model.predict(X_test_scaled)
y_pred_proba_logistic = logistic_model.predict_proba(X_test_scaled)[:, 1]

# 2. Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=10)
rf_model.fit(X_train, y_train)

y_pred_rf = rf_model.predict(X_test)
y_pred_proba_rf = rf_model.predict_proba(X_test)[:, 1]

# Model Evaluation
print("=== MODEL PERFORMANCE ===")
print(f"Logistic Regression Accuracy: {accuracy_score(y_test, y_pred_logistic):.3f}")
print(f"Random Forest Accuracy: {accuracy_score(y_test, y_pred_rf):.3f}")

# Feature importance from Random Forest
feature_importance = pd.DataFrame({
    'feature': feature_columns,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

print("\nRandom Forest Feature Importance:")
print(feature_importance)

# Logistic Regression Coefficients
log_coefficients = pd.DataFrame({
    'feature': feature_columns,
    'coefficient': logistic_model.coef_[0],
    'abs_coefficient': np.abs(logistic_model.coef_[0])
}).sort_values('abs_coefficient', ascending=False)

print("\nLogistic Regression Coefficients:")
print(log_coefficients)

# Classification Reports
print("\n=== LOGISTIC REGRESSION CLASSIFICATION REPORT ===")
print(classification_report(y_test, y_pred_logistic))

print("\n=== RANDOM FOREST CLASSIFICATION REPORT ===")
print(classification_report(y_test, y_pred_rf))

In [ ]:
# Correlation Analysis and Statistical Tests

from scipy.stats import pearsonr, spearmanr, chi2_contingency

print("=== CORRELATION ANALYSIS ===")

# Economic correlations with round outcomes
economic_correlations = {}
for feature in [ 'spending_advantage', 'equipment_advantage']:
    corr_coef, p_value = pearsonr(model_data[feature], model_data['team1_winner'])
    economic_correlations[feature] = {'correlation': corr_coef, 'p_value': p_value}
    print(f"{feature}: r={corr_coef:.3f}, p={p_value:.3f}")

# Performance correlations
performance_correlations = {}
for feature in ['kill_advantage']:
    corr_coef, p_value = pearsonr(model_data[feature], model_data['team1_winner'])
    performance_correlations[feature] = {'correlation': corr_coef, 'p_value': p_value}
    print(f"{feature}: r={corr_coef:.3f}, p={p_value:.3f}")

# Side advantage analysis
ct_analysis = model_data.groupby('is_ct_t1')['team1_winner'].agg(['count', 'mean', 'std'])
print(f"\n=== SIDE ANALYSIS ===")
print("Team 1 performance by side:")
print(ct_analysis)

# Round type analysis
round_type_analysis = model_data.groupby(['is_pistol_round', 'is_eco_round'])['team1_winner'].agg(['count', 'mean'])
print(f"\n=== ROUND TYPE ANALYSIS ===")
print("Win rates by round type:")
print(round_type_analysis)





## 9. Summary Visualization Functions

In [ ]:
# Visualization Functions for Key Insights

def create_summary_plots():
    """Create comprehensive summary visualizations"""
    
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    fig.suptitle('CS:GO Analytics Summary Dashboard', fontsize=16, fontweight='bold')
    
    # 1. Economic Distribution
    if 'player_economics' in globals():
        axes[0,0].hist(player_economics['avg_money_spent'], bins=30, alpha=0.7, edgecolor='black')
        axes[0,0].set_title('Player Spending Distribution')
        axes[0,0].set_xlabel('Average Money Spent per Round')
        axes[0,0].set_ylabel('Frequency')
    
    # 2. KDA Performance
    if 'kda_data' in globals():
        axes[0,1].scatter(kda_data['total_kills'], kda_data['kd_ratio'], alpha=0.6)
        axes[0,1].set_title('Kill Performance vs KD Ratio')
        axes[0,1].set_xlabel('Total Kills')
        axes[0,1].set_ylabel('KD Ratio')
    
    # 3. Economic Advantage vs Win Rate
    if 'team_data' in globals():
        win_by_advantage = team_data.groupby(pd.cut(team_data['spending_advantage'], bins=10))['team1_winner'].mean()
        win_by_advantage.plot(kind='bar', ax=axes[0,2], rot=45)
        axes[0,2].set_title('Win Rate by Spending Advantage')
        axes[0,2].set_ylabel('Win Rate')
    
    # 4. Weapon Efficiency
    if 'weapon_efficiency' in globals():
        top_weapons_plot = weapon_efficiency.nlargest(8, 'total_kills')
        axes[1,0].barh(range(len(top_weapons_plot)), top_weapons_plot['headshot_rate'])
        axes[1,0].set_yticks(range(len(top_weapons_plot)))
        axes[1,0].set_yticklabels(top_weapons_plot['weapon'])
        axes[1,0].set_title('Headshot Rate by Weapon')
        axes[1,0].set_xlabel('Headshot Rate')
    
    # 5. Round End Reasons
    if 'ways_data' in globals():
        reason_counts = ways_data.groupby('reason_name')['frequency'].sum().nlargest(6)
        reason_counts.plot(kind='pie', ax=axes[1,1], autopct='%1.1f%%')
        axes[1,1].set_title('Round End Distribution')
    
    # 6. Feature Importance (if available)
    if 'feature_importance' in globals():
        top_features = feature_importance.head(8)
        axes[1,2].barh(range(len(top_features)), top_features['importance'])
        axes[1,2].set_yticks(range(len(top_features)))
        axes[1,2].set_yticklabels(top_features['feature'])
        axes[1,2].set_title('Most Important Features for Win Prediction')
        axes[1,2].set_xlabel('Importance')
    
    plt.tight_layout()
    plt.show()

def print_executive_summary():
    """Print executive summary of all analyses"""
    
    print("="*80)
    print("CS:GO ANALYTICS EXECUTIVE SUMMARY")
    print("="*80)
    
    # Economic insights
    if 'player_economics' in globals():
        print(f"\n📊 ECONOMIC INSIGHTS:")
        print(f"   • Average spending per round: ${player_economics['avg_money_spent'].mean():.0f}")
        print(f"   • Spending volatility (std): ${player_economics['avg_money_spent'].std():.0f}")
        print(f"   • Top 10% spenders threshold: ${player_economics['avg_money_spent'].quantile(0.9):.0f}")
    
    # Performance insights  
    if 'kda_data' in globals():
        print(f"\n🎯 PERFORMANCE INSIGHTS:")
        print(f"   • Average KD ratio: {kda_data['kd_ratio'].mean():.2f}")
        print(f"   • Best performer: {kda_data.loc[kda_data['kd_ratio'].idxmax(), 'player_hltv_name']}")
        print(f"   • Average damage per round: {kda_data['avg_damage_per_round'].mean():.0f}")
    
    # Weapon insights
    if 'weapon_efficiency' in globals():
        print(f"\n🔫 WEAPON INSIGHTS:")
        print(f"   • Overall headshot rate: {weapon_efficiency['headshot_rate'].mean():.1%}")
        most_used = weapon_efficiency.loc[weapon_efficiency['total_kills'].idxmax(), 'weapon']
        print(f"   • Most lethal weapon: {most_used}")
    
    # Economic correlation
    if 'economic_correlations' in globals():
        print(f"\n💰 ECONOMIC CORRELATIONS:")
        for feature, stats in economic_correlations.items():
            print(f"   • {feature}: r={stats['correlation']:.3f} (p={stats['p_value']:.3f})")
    
    # Model performance
    if 'y_test' in globals() and 'y_pred_rf' in globals():
        print(f"\n🤖 PREDICTIVE MODEL:")
        print(f"   • Random Forest accuracy: {accuracy_score(y_test, y_pred_rf):.1%}")
        print(f"   • Baseline (majority class): {max(y_test.mean(), 1-y_test.mean()):.1%}")
    
    print(f"\n" + "="*80)

# Create summary functions that can be called
print("📈 Summary visualization and reporting functions created!")
print("Run create_summary_plots() and print_executive_summary() to see results.")